In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATAPATH = "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer/data/"

In [ ]:
import pandas as pd
import os
#!pip install https://huggingface.co/kormilitzin/en_core_med7_lg.tar.gz
!pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl
!pip install -U spacy==3.2.0
#import spacy.cli
#spacy.cli.download("en_core_med7_lg")
#spacy.cli.download("en_core_med7_trf")
import spacy
#import en_core_med7_lg
med7 = spacy.load('en_core_med7_lg')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.4/607.4 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 52.7 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.1
    Uninstalling spacy-3.5.1:
      Successfully uninstalled spacy-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.4.4 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 51.8 MB/s e

/usr/local/lib/python3.9/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_med7_lg' (3.4.2.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.2.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
preprocessed_df = pd.read_pickle(os.path.join(DATAPATH, "preprocessed_notes_los.p"))

In [ ]:
preprocessed_df['ner'] = None

In [ ]:
print(len(preprocessed_df))

181483


In [ ]:
count = 0
preprocessed_index = {}
for i in preprocessed_df.itertuples():
    
    if count % 1000 == 0:
        print(count)
        #if count == 1000:
        #    break  

    count += 1
    ind = i.Index
    text = i.preprocessed_text
    
    all_pred = []
    for each_sent in text:
        try:
            doc = med7(each_sent)
            result = ([(ent.text, ent.label_) for ent in doc.ents])
            if len(result) == 0: continue
            all_pred.append(result)
        except:
            print("error..")
            continue
    preprocessed_df.at[ind, 'ner'] = all_pred

In [ ]:
#pd.to_pickle(preprocessed_df, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/ner_df_final.p")
pd.to_pickle(preprocessed_df, (os.path.join(DATAPATH, "ner_df_final_los.p")))
print("Completed Writing Final LOS File")